In [1]:
import os, glob
import numpy as np
import random
import copy

In [2]:
files = glob.glob('/home/fouchee/git/outlier-documents/data/reuters21578/*.sgm')

In [3]:
files

['/home/fouchee/git/outlier-documents/data/reuters21578/reut2-003.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-015.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-002.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-009.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-020.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-011.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-010.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-019.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-012.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-007.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-018.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-017.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-021.sgm',
 '/home/fouchee/git/outlier-documents/data/reuters21578/reut2-00

In [4]:
f = open('/home/fouchee/git/outlier-documents/data/reuters21578/reut2-003.sgm')

In [5]:
lines = []
for file in files:
    f = open(file, errors="ignore")
    lines = lines + f.readlines()

In [6]:
documents = dict()

i = 0
for line in lines:
    if(line.startswith("<REUTERS")):
        documents[i] = documents.get(i, "") + line
    elif(line.startswith("</REUTERS")):
        documents[i] = documents.get(i, "") + line
        i += 1
    else:
        documents[i] = documents.get(i, "") + line

In [7]:
topics = dict()
bodies = dict()

for key in documents.keys():
    text = documents[key]
    body = text[text.find("<BODY>")+6:text.find("</BODY>")]
    top = text[text.find("<TOPICS>")+8:text.find("</TOPICS>")]
    
    topics[key] = top
    bodies[key] = body

In [8]:
len(documents.keys())

21578

In [9]:
# do not consider those with multiple topics
for key in topics.keys():
    if "><" in topics[key]:
        topics[key] = ''

In [10]:
inliers = dict() # there are 6416, i.e., not matching really the reference 
outlierpool = dict()

for key in documents.keys(): 
    if "earn" in topics[key]:
        inliers[key] = bodies[key]
    elif "acq" in topics[key]:
        inliers[key] = bodies[key]
    elif "interest" in topics[key]:
        outlierpool[key] = bodies[key]
    

In [11]:
len(inliers.keys()) # still not matching really the reference 5768

6307

In [12]:
inlierpaths = []
outlierpaths = []

for key in inliers.keys(): 
    path = "data/1.0/earnacq%s"%key
    f = open(path, "w")
    f.write(inliers[key])
    f.close()
    inlierpaths = inlierpaths + [path]
    
for key in outlierpool.keys(): 
    path = "data/1.0/interest%s"%key
    f = open(path, "w")
    f.write(outlierpool[key])
    f.close()
    outlierpaths = outlierpaths + [path]

In [13]:
nbenchmarks = 10

In [14]:
for x in range(nbenchmarks):
    benchmark = open("reuters.%s.data"%(x+1), 'w')
    
    outliersamples = random.sample(outlierpaths,100)
    tmpinlierpaths = copy.deepcopy(inlierpaths)
    
    positions = [0 for x in range(len(tmpinlierpaths))] + [1 for x in range(100)]
    
    random.shuffle(positions)
    random.shuffle(tmpinlierpaths)
    random.shuffle(outliersamples)
    
    for x in positions:
        if x == 1:
            file = outliersamples.pop()
            benchmark.write("bm/reuters/" + file + "\t" + "1\n")
        else:
            file = tmpinlierpaths.pop()
            benchmark.write("bm/reuters/" + file + "\t" + "0\n")
    benchmark.close()